In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
import seaborn as sns
import platform

# seaborn 설정 리셋
sns.reset_defaults()

# 폰트설정
if platform.system() == 'Windows' :
    path = 'c:/Windows/Fonts/malgun.ttf'
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
elif platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
else :
    print('Check your OS System')
    
# 그래프에 마이너스 표시
matplotlib.rcParams['axes.unicode_minus'] = False

## age 결측값 어린이 ,성인, 노인 분류 하기

In [2]:
# 데이터 불러오기

### <변수설명>
# PassengerId : 각 승객의 고유 번호
# Survived : 생존 여부(종속 변수)
#     0 = 사망
#     1 = 생존
# Pclass : 객실 등급 - 승객의 사회적, 경제적 지위
#     1st = Upper(first class)
#     2nd = Middle(second class)
#     3rd = Lower(third class)
# Name : 이름
# Sex : 성별
# Age : 나이
# SibSp : 동반한 Sibling(형제자매)와 Spouse(배우자)의 수
# Parch : 동반한 Parent(부모) Child(자식)의 수
# Ticket : 티켓의 고유넘버
# Fare : 티켓의 요금
# Cabin : 객실 번호
# Embarked : 승선한 항
#     C = Cherbourg
#     Q = Queenstown
#     S = Southampton

In [3]:
# 데이터불러오기
titanic = pd.read_csv('./data/06_(문제)타이타닉_분류_데이터셋.csv')
titanic

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


In [4]:
# 데이터전처리 - 결측치 확인
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   survived     891 non-null    int64  
 1   pclass       891 non-null    int64  
 2   sex          891 non-null    object 
 3   age          714 non-null    float64
 4   sibsp        891 non-null    int64  
 5   parch        891 non-null    int64  
 6   fare         891 non-null    float64
 7   embarked     889 non-null    object 
 8   class        891 non-null    object 
 9   who          891 non-null    object 
 10  adult_male   891 non-null    bool   
 11  deck         203 non-null    object 
 12  embark_town  889 non-null    object 
 13  alive        891 non-null    object 
 14  alone        891 non-null    bool   
dtypes: bool(2), float64(2), int64(4), object(7)
memory usage: 92.4+ KB


In [5]:
# 데이터전처리 - 결측치 처리 
# 결측치가 너무 많은 deck컬럼 제거 
titanic= titanic.drop(columns='deck')

In [6]:
# 데이터 전처리 - 결측치 처리
# embarked의 결측치 2건 > 머신러닝을 통해 분류한 값 'S' 입력
titanic['embarked'].fillna('S', inplace=True)

In [7]:
# one_hot 인코딩을 통한 데이터 정형화
titanic_one_hot = pd.get_dummies(titanic)

In [8]:
# True, False > 1, 0 치환
titanic_one_hot[['adult_male','alone']] = titanic_one_hot[['adult_male','alone']].astype(int)

In [9]:
# 최종 결측치 확인
titanic_one_hot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   survived                 891 non-null    int64  
 1   pclass                   891 non-null    int64  
 2   age                      714 non-null    float64
 3   sibsp                    891 non-null    int64  
 4   parch                    891 non-null    int64  
 5   fare                     891 non-null    float64
 6   adult_male               891 non-null    int32  
 7   alone                    891 non-null    int32  
 8   sex_female               891 non-null    uint8  
 9   sex_male                 891 non-null    uint8  
 10  embarked_C               891 non-null    uint8  
 11  embarked_Q               891 non-null    uint8  
 12  embarked_S               891 non-null    uint8  
 13  class_First              891 non-null    uint8  
 14  class_Second             8

In [10]:
# 예측데이터 분리
predict = titanic_one_hot[titanic_one_hot['age'].isna()].drop(columns='age')

In [11]:
# 새로운 변수에 저장
age_change = titanic_one_hot

In [12]:
# 성인, 노인 분리
age_change['age_type'] = (age_change['age'] >= 16) & (age_change['age'] <= 49)

In [13]:
age_change['age_type'] = age_change['age_type'].astype(int)

In [14]:
# 어린이는 who컬럼 확인결과 결측치에 포함되지않기 때문에 분리
age_change = age_change[age_change['age'] >= 15]

In [15]:
# 종속변수 지정
target = age_change[age_change['age'].isna()==False]['age_type'].to_numpy()

In [16]:
# 독립변수 지정
data = age_change[age_change['age'].isna()==False].drop(columns=['age','age_type']).to_numpy()

In [17]:
# 분리 확인
print(data.shape, target.shape)

(636, 23) (636,)


In [18]:
# 훈련데이터, 테스트데이터 분리
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = \
    train_test_split(data, target, random_state= 111, stratify= target)

In [19]:
# 분리확인
print(train_input.shape, train_target.shape)
print(test_input.shape, test_target.shape)

(477, 23) (477,)
(159, 23) (159,)


In [20]:
# 정규화
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
ss.fit(train_input, train_target)

train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)

In [21]:
# 하이퍼파라미터 튜닝 모델 불러오기
from sklearn.model_selection import GridSearchCV

In [22]:
# 교차검증 모델 불러오기
from sklearn.model_selection import cross_validate

## 1. KNN모델

In [23]:
# KNN 모델 불러오기
from sklearn.neighbors import KNeighborsClassifier

In [24]:
# 기본 데이터 훈련 및 정확도 확인
kn = KNeighborsClassifier()
kn.fit(train_input, train_target)

print('<KNN> 기본   훈련데이터 정확도 :', round(kn.score(train_input, train_target),4))
print('<KNN> 기본 테스트데이터 정확도 :', round(kn.score(test_input, test_target),4))

<KNN> 기본   훈련데이터 정확도 : 0.891
<KNN> 기본 테스트데이터 정확도 : 0.8742


In [25]:
# 정규화 데이터 훈련 및 정확도 확인
kn = KNeighborsClassifier()
kn.fit(train_scaled, train_target)

print('<KNN> 기본   훈련데이터 정확도 :', round(kn.score(train_scaled, train_target),4))
print('<KNN> 기본 테스트데이터 정확도 :', round(kn.score(test_scaled, test_target),4))

<KNN> 기본   훈련데이터 정확도 : 0.891
<KNN> 기본 테스트데이터 정확도 : 0.8805


In [57]:
# 하이퍼파라미터 튜닝 및 교차검증을 통한 정확도 확인
kn = KNeighborsClassifier()

# 그리드 서치를 활용한 최적의 이웃갯수 값 탐색
params = {'n_neighbors' : range(3, 51, 2)}
gs = GridSearchCV(kn, params, n_jobs=-1)
gs.fit(train_input, train_target)

# 최적의 이웃갯수 값을 적용한 모델 생성
kn = gs.best_estimator_

# 교차검증을 통한 정확도 확인
scores = cross_validate(kn, train_input, train_target,
                        return_train_score=True, n_jobs=-1)

print('<KNN> 튜닝   훈련데이터 정확도 :', round(scores['train_score'].mean(),4))
print('<KNN> 튜닝 테스트데이터 정확도 :', round(scores['test_score'].mean(),4))

<KNN> 튜닝   훈련데이터 정확도 : 0.8779
<KNN> 튜닝 테스트데이터 정확도 : 0.8763


## 2.로지스틱 회귀분류모델

In [58]:
# 로지스틱 회귀분류모델 불러오기
from sklearn.linear_model import LogisticRegression

# 반복훈련횟수 설정 (독립변수의 갯수 * 100(기본값))
max_iter = len(titanic_one_hot.columns) * 100

In [59]:
# 기본 데이터를 통한 모델 훈련 및 정확도 확인 (하이퍼파라미터 : max_iter = 독립변수의 갯수 * 100(기본값))
lr = LogisticRegression(max_iter= max_iter)

lr.fit(train_input, train_target)

print('<로지스틱> 기본   훈련데이터 정확도 :', round(lr.score(train_input, train_target),4))
print('<로지스틱> 기본 테스트데이터 정확도 :', round(lr.score(test_input, test_target),4))

<로지스틱> 기본   훈련데이터 정확도 : 0.8805
<로지스틱> 기본 테스트데이터 정확도 : 0.8742


In [60]:
# 기본 데이터를 통한 모델 훈련 및 정확도 확인 (하이퍼파라미터 : max_iter = 독립변수의 갯수 * 100(기본값))
lr = LogisticRegression(max_iter = max_iter)

lr.fit(train_scaled, train_target)

print('<로지스틱> 정규화   훈련데이터 정확도 :', round(lr.score(train_scaled, train_target),4))
print('<로지스틱> 정규화 테스트데이터 정확도 :', round(lr.score(test_scaled, test_target),4))

<로지스틱> 정규화   훈련데이터 정확도 : 0.8847
<로지스틱> 정규화 테스트데이터 정확도 : 0.8805


In [61]:
# 하이퍼파라미터 튜닝 및 교차검증을 통한 정확도 확인
lr = lr = LogisticRegression()

# 그리드 서치를 활용한 최적의 가중치, 훈련반복횟수 값 탐색
params = {'C' : np.arange(0.001, 0.01, 0.001),
          'max_iter' : range(3000, 5000, 1000)}

gs = GridSearchCV(lr, params, n_jobs=-1)
gs.fit(train_input, train_target)

# 최적의 이웃갯수 값을 적용한 모델 생성
lr = gs.best_estimator_

# # 교차검증을 통한 정확도 확인
scores = cross_validate(lr, train_input, train_target,
                        return_train_score=True, n_jobs=-1)

print('<로지스틱> 튜닝   훈련데이터 정확도 :', round(scores['train_score'].mean(),4))
print('<로지스틱> 튜닝 테스트데이터 정확도 :', round(scores['test_score'].mean(),4))

<로지스틱> 튜닝   훈련데이터 정확도 : 0.8753
<로지스틱> 튜닝 테스트데이터 정확도 : 0.8721


In [31]:
# 랜덤포레스트 모델 불러오기
from sklearn.ensemble import RandomForestClassifier

In [71]:
# 기본 데이터를 통한 모델 훈련 및 정확도 확인(하이퍼파라미터 : random_state = 42)
rf = RandomForestClassifier(n_jobs=-1, random_state=111)

rf.fit(train_input, train_target)

print('<랜덤포레스트> 기본   훈련데이터 정확도 :', round(rf.score(train_input, train_target),4))
print('<랜덤포레스트> 기본 테스트데이터 정확도 :', round(rf.score(test_input, test_target),4))

<랜덤포레스트> 기본   훈련데이터 정확도 : 0.9686
<랜덤포레스트> 기본 테스트데이터 정확도 : 0.8491


In [72]:
# 정규화 데이터를 통한 모델 훈련 및 정확도 확인(하이퍼파라미터 : random_state = 42)
rf = RandomForestClassifier(n_jobs=-1, random_state=111)

rf.fit(train_scaled, train_target)

print('<랜덤포레스트> 정규화   훈련데이터 정확도 :', round(rf.score(train_scaled, train_target),4))
print('<랜덤포레스트> 정규화 테스트데이터 정확도 :', round(rf.score(test_scaled, test_target),4))

<랜덤포레스트> 정규화   훈련데이터 정확도 : 0.9686
<랜덤포레스트> 정규화 테스트데이터 정확도 : 0.8491


In [73]:
# 교차검증을 통한 모델 훈련 및 정확도 확인(하이퍼파라미터 : random_state = 42)
rf = RandomForestClassifier(n_jobs=-1, random_state=111)

scores = cross_validate(rf, train_input, train_target,
                        return_train_score=True, n_jobs=-1)

print('<랜덤포레스트> 교차검증   훈련데이터 정확도 :', round(scores['train_score'].mean(),4))
print('<랜덤포레스트> 교차검증 테스트데이터 정확도 :', round(scores['test_score'].mean(),4))

<랜덤포레스트> 교차검증   훈련데이터 정확도 : 0.9691
<랜덤포레스트> 교차검증 테스트데이터 정확도 : 0.8554


In [35]:
# 그레디언트부스팅 모델 불러오기
from sklearn.ensemble import GradientBoostingClassifier

In [74]:
# 기본 데이터를 통한 모델 훈련 및 정확도 확인(하이퍼파라미터 : random_state = 42)
gb = GradientBoostingClassifier(random_state=111)

gb.fit(train_input, train_target)

print('<그레디언트부스팅> 기본   훈련데이터 정확도 :', round(gb.score(train_input, train_target),4))
print('<그레디언트부스팅> 기본 테스트데이터 정확도 :', round(gb.score(test_input, test_target),4))

<그레디언트부스팅> 기본   훈련데이터 정확도 : 0.9329
<그레디언트부스팅> 기본 테스트데이터 정확도 : 0.8742


In [75]:
# 정규화 데이터를 통한 모델 훈련 및 정확도 확인(하이퍼파라미터 : random_state = 42)
gb = GradientBoostingClassifier(random_state=111)

gb.fit(train_scaled, train_target)

print('<그레디언트부스팅> 정규화   훈련데이터 정확도 :', round(gb.score(train_scaled, train_target),4))
print('<그레디언트부스팅> 정규화 테스트데이터 정확도 :', round(gb.score(test_scaled, test_target),4))

<그레디언트부스팅> 정규화   훈련데이터 정확도 : 0.9329
<그레디언트부스팅> 정규화 테스트데이터 정확도 : 0.8742


In [78]:
# 교차검증을 통한 모델 훈련 및 정확도 확인(하이퍼파라미터 : random_state = 42)
gb = GradientBoostingClassifier(random_state=111)

scores = cross_validate(gb, train_input, train_target,
                        return_train_score=True, n_jobs=-1)

print('<그레디언트부스팅> 교차검증   훈련데이터 정확도 :', round(scores['train_score'].mean(),4))
print('<그레디언트부스팅> 교차검증 테스트데이터 정확도 :', round(scores['test_score'].mean(),4))

<그레디언트부스팅> 교차검증   훈련데이터 정확도 : 0.9382
<그레디언트부스팅> 교차검증 테스트데이터 정확도 : 0.8804


In [77]:
# 히스토그램기반 그레디언트부스팅 모델 불러오기
from sklearn.ensemble import HistGradientBoostingClassifier

In [79]:
# 기본 데이터를 통한 모델 훈련 및 정확도 확인(하이퍼파라미터 : random_state = 42)
hgb = HistGradientBoostingClassifier(random_state=42)

hgb.fit(train_input, train_target)

print('<히스토그램기반 그레디언트부스팅> 기본   훈련데이터 정확도 :', round(hgb.score(train_input, train_target),4))
print('<히스토그램기반 그레디언트부스팅> 기본 테스트데이터 정확도 :', round(hgb.score(test_input, test_target),4))

<히스토그램기반 그레디언트부스팅> 기본   훈련데이터 정확도 : 0.935
<히스토그램기반 그레디언트부스팅> 기본 테스트데이터 정확도 : 0.8491


In [80]:
# 정규화 데이터를 통한 모델 훈련 및 정확도 확인(하이퍼파라미터 : random_state = 42)
hgb = HistGradientBoostingClassifier(random_state=42)

hgb.fit(train_scaled, train_target)

print('<히스토그램기반 그레디언트부스팅> 정규화   훈련데이터 정확도 :', round(hgb.score(train_scaled, train_target),4))
print('<히스토그램기반 그레디언트부스팅> 정규화 테스트데이터 정확도 :', round(hgb.score(test_scaled, test_target),4))

<히스토그램기반 그레디언트부스팅> 정규화   훈련데이터 정확도 : 0.935
<히스토그램기반 그레디언트부스팅> 정규화 테스트데이터 정확도 : 0.8491


In [81]:
# 교차검증을 통한 모델 훈련 및 정확도 확인(하이퍼파라미터 : random_state = 42)
hgb = HistGradientBoostingClassifier(random_state=42)

scores = cross_validate(hgb, train_input, train_target,
                        return_train_score=True, n_jobs=-1)

print('<히스토그램기반 그레디언트부스팅> 교차검증   훈련데이터 정확도 :', round(scores['train_score'].mean(),4))
print('<히스토그램기반 그레디언트부스팅> 교차검증 테스트데이터 정확도 :', round(scores['test_score'].mean(),4))

<히스토그램기반 그레디언트부스팅> 교차검증   훈련데이터 정확도 : 0.9345
<히스토그램기반 그레디언트부스팅> 교차검증 테스트데이터 정확도 : 0.8762


In [82]:
# <최종모델선정>
# 교차검증을 통한 그레디언트부스팅 모델

# <최종모델 정확도>
# <그레디언트부스팅> 교차검증   훈련데이터 정확도 : 0.9382
# <그레디언트부스팅> 교차검증 테스트데이터 정확도 : 0.8804

# <선정이유>
# 전체적인 정확도가 좋으며 일반화의 정도가 0.05 정도로 크지 않기때문에 선정

In [85]:
# <선정모델을 통한 예측결과>
gb.fit(train_input, train_target)
gb.predict(predict)

# 0 = 노인, 1 = 성인
# 1명의 노인 승객 확인

C:\Users\admin\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but GradientBoostingClassifier was fitted without feature names
  warnings.warn(


array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1])